[참고](https://github.com/rickiepark/handson-ml2/blob/master/12_custom_models_and_training_with_tensorflow.ipynb)

# 12.1 텐서플로 훑어보기
텐서플로가 제공하는 것
- GPU 지원
- 분산 컴퓨팅 지원
- JIT(just-in-time) 컴파일러를 포함. 파이썬 함수에서 **계산 그래프**(computation graph)를 추출한 다음 최적화하고 효율적으로 실행
- 훈련한 환경과 다른 환경에서 실행할 수 있음
- 자동 미분(autodiff) 기능과 RMSProp, Nadam 같은 고성능 옵티마이저 제공

# 12.2 넘파이처럼 텐서플로 사용하기
## 12.2.1 텐서와 연산


In [1]:
import tensorflow as tf

tf.constant([ [1., 2., 3.], [4., 5., 6.]]) # 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [2]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [3]:
t = tf.constant([ [1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [4]:
t.dtype

tf.float32

In [5]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [6]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [7]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [8]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [9]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [10]:
tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [11]:
t.__add__(10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

### 케라스의 저수준 API
`keras.backend`를 사용하는 예. 보통 별칭 `K`를 사용함.

In [12]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

## 12.2.2 텐서와 넘파이

In [13]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

넘파이는 기본으로 63비트 정밀도를 사용하지만 텐서플로는 32비트 정밀도를 사용하기 때문에 <br/> 
넘파이 배열로 텐서를 만들 때 `dtype=tf.float32`로 지정해야 함.

## 12.2.3 타입 변환
텐서플로는 어떤 타입 변환도 자동으로 수행하지 않는다.

In [17]:
# tf.constant(2.) + tf.constant(40) <--- 이렇게 하면 에러 남. 실수와 정수를 더할 수 없음.
# tf.constant(2.) + tf.constant(40., dtype=tf.float64) <--- 이렇게 해도 에러 남. 32비트와 64비트도 더할 수 없음.
tf.constant(2.) + tf.constant(40.) # 이렇게 해야 에러가 안 남!

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [18]:
# 타입 변환이 필요할 때는 `tf.cast()` 사용하면 됨!
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 12.2.4 변수
`tf.Tensor`는 변경이 불가능하기 때문에 변경되어야 하는 파라미터가 필요하면 `tf.Variable`를 사용하자.

In [19]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [20]:
# assign 메서드를 사용하여 변숫값을 바꿀 수도 있음.
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [21]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [23]:
# scatter_nd_update() 메서드로 개별 원소를 수정할 수도 있음.
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# 12.3 사용자 정의 모델과 훈련 알고리즘
## 12.3.1 사용자 정의 손실 함수
회귀 모델을 훈련하는 데 훈련 세트에 잡음 데이터가 조금 있을 때 후버(Huber) 손실을 사용하면 좋다.

In [24]:
def huber_fn(y_true, y_pred) :
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [29]:
# 캘리포니아 주택 가격으로 훈련
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [30]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [32]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [33]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 1.0121 - mae: 1.4161 - val_loss: 0.3403 - val_mae: 0.6367
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2151 - mae: 0.5147 - val_loss: 0.2838 - val_mae: 0.5721


## 12.3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기

In [35]:
model.save("my_model_with_a_custom_loss.h5")

In [37]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                               custom_objects={"huber_fn" : huber_fn})

In [39]:
def create_huber(threshold=1.0) : 
    def huber_fn(y_true, y_pred) :
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold*tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn
model.compile(loss=create_huber(2.0), optimizer="nadam")

`threshold` 값은 저장되지 않기 때문에 모델을 로드할 때 `threshold` 값을 지정해야 함.

In [40]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [41]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [42]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs): 
        self.threshold = threshold
        super().__init__(**kwargs) # 이렇게 받은 값을 부모 클래스의 생성자에게 전달.
    def call(self, y_true, y_pred): # 레이블과 예측을 받고 모든 샘플의 손실을 계산.
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self): # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리를 반환.
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [43]:
# 모델을 컴파일할 때 이 클래스의 인스턴스를 사용할 수 있음.
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [45]:
model.save("my_model_with_a_custom_loss_class.h5")

In [46]:
# 모델을 저장할 때 임곗값도 저장됨.
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", # TODO: check PR #25956
                               custom_objects={"HuberLoss": HuberLoss})

## 12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [47]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [48]:
def my_softplus(z): # tf.nn.softplus(z) 값을 반환.
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # tf.nn.relu(weights) 값을 반환.
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [49]:
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [50]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [51]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}